In [6]:
from human_feedback_api import Comparison

experiment_name = '1hoop_hum_2017-11-20-2255'

Comparison.objects.filter(experiment_name=experiment_name).count()

917

In [14]:
import sqlite3
conn = sqlite3.connect('/Users/cristian/dev/workspace/cs229/cs229-proj/models/1hoop/db.sqlite3')
c = conn.cursor()
c.execute('SELECT * From Comparisons')

OperationalError: no such table: Comparisons